In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from collections import Counter
import datetime
import missingno as ms
from scipy import stats
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [22]:
transactions_1 = pd.read_csv('transactions.csv') 
transactions = transactions_1.append(pd.read_csv('transactions_v2.csv') )
user_test = pd.read_csv('Jan_tran_id_expire.csv')
user_validation = pd.read_csv('Feb_tran_id_expire.csv')

In [ ]:
def memory_reduce_int(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127)&(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767)&(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647)&(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
            
def memory_reduce_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)
        
def memory_usage(df):
    memory = df.memory_usage(index=True).sum()
    print(memory/ 1024**2," MB")

def memory_reduce(df):
    memory_usage(df)
    memory_reduce_int(df)
    memory_reduce_float(df)
    memory_usage(df)

In [5]:
transactions.drop_duplicates(inplace=True)

In [7]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [15]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], format='%Y%m%d')
transactions['membership_expire_date'] = pd.to_datetime(transactions['membership_expire_date'], format='%Y%m%d')

In [16]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22975416 entries, 0 to 1431008
Data columns (total 9 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   msno                    object        
 1   payment_method_id       int64         
 2   payment_plan_days       int64         
 3   plan_list_price         int64         
 4   actual_amount_paid      int64         
 5   is_auto_renew           int64         
 6   transaction_date        datetime64[ns]
 7   membership_expire_date  datetime64[ns]
 8   is_cancel               int64         
dtypes: datetime64[ns](2), int64(6), object(1)
memory usage: 1.7+ GB


In [17]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,1970-01-01 00:00:00.020150930,1970-01-01 00:00:00.020151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,1970-01-01 00:00:00.020150930,1970-01-01 00:00:00.020151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,1970-01-01 00:00:00.020150930,1970-01-01 00:00:00.020160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,1970-01-01 00:00:00.020150930,1970-01-01 00:00:00.020151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,1970-01-01 00:00:00.020150930,1970-01-01 00:00:00.020151121,0


In [19]:
user_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990745 entries, 0 to 990744
Data columns (total 2 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   msno                    990745 non-null  object
 1   membership_expire_date  990745 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.1+ MB


In [26]:
user_test['membership_expire_date'] = pd.to_datetime(user_test['membership_expire_date'], format='%Y%m%d')

In [29]:
user_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990745 entries, 0 to 990744
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   msno                    990745 non-null  object        
 1   membership_expire_date  990745 non-null  datetime64[ns]
 2   date_prior_90           990745 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 22.7+ MB


In [31]:
user_test['date_prior_90']=user_test['membership_expire_date'] - pd.DateOffset(months=3)

In [33]:
train_transaction = pd.merge(user_test, transactions, on = 'msno', how = 'left')

In [42]:
train_transaction

,msno,membership_expire_date_x,date_prior_90,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date_y,is_cancel
0,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2016-09-30,2016-10-31,0
1,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2016-07-31,2016-08-31,0
2,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2015-10-31,2015-11-30,0
3,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2017-02-28,2017-03-31,0
4,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2016-06-30,2016-07-31,0
...,...,...,...,...,...,...,...,...,...,...,...
17219293,1hapHdS7PNnKCRXnUp4X+5p//irxpsojYZB95dIndq0=,2017-01-01,2016-10-01,41,30,149,149,1,2016-12-31,2017-02-01,0
17219294,1hapHdS7PNnKCRXnUp4X+5p//irxpsojYZB95dIndq0=,2017-01-01,2016-10-01,41,30,149,149,1,2015-06-30,2015-08-01,0
17219295,1hapHdS7PNnKCRXnUp4X+5p//irxpsojYZB95dIndq0=,2017-01-01,2016-10-01,41,30,149,149,1,2015-03-31,2015-05-01,0
17219296,1hapHdS7PNnKCRXnUp4X+5p//irxpsojYZB95dIndq0=,2017-01-01,2016-10-01,41,30,149,149,1,2017-02-28,2017-04-01,0


In [41]:
train_transaction['transaction_date']=pd.to_datetime(train_transaction['transaction_date'], format='%Y%m%d')
train_transaction['membership_expire_date_y']=pd.to_datetime(train_transaction['membership_expire_date_y'], format='%Y%m%d')

In [43]:
train_transaction=train_transaction[(train_transaction['transaction_date']>train_transaction['date_prior_90'])&(train_transaction['transaction_date']<train_transaction['membership_expire_date_x'])]

In [46]:
train_transaction.drop('membership_expire_date_y',1,inplace=True)

In [47]:
train_transaction

,msno,membership_expire_date_x,date_prior_90,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,is_cancel
11,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2016-12-31,0
13,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,41,30,99,99,1,2016-11-30,0
21,vSZ4aJziXuMmmm4wKMVTg3plvNspkVlJ1lZH1lsUd50=,2017-01-31,2016-10-31,34,30,149,149,1,2016-11-30,0
31,vSZ4aJziXuMmmm4wKMVTg3plvNspkVlJ1lZH1lsUd50=,2017-01-31,2016-10-31,34,30,149,149,1,2016-12-31,0
46,/w9E+Tz8Ny3anC3hVFy8iLXGdWFoQ13fmwKjjd2daLg=,2017-01-31,2016-10-31,31,30,149,149,1,2016-11-30,0
...,...,...,...,...,...,...,...,...,...,...
17219253,pJzUz1g5hC/tHN3/DZsjF0l8j5zb3Tykmc4HJMv98QY=,2017-01-01,2016-10-01,41,30,99,99,1,2016-12-01,0
17219261,pJzUz1g5hC/tHN3/DZsjF0l8j5zb3Tykmc4HJMv98QY=,2017-01-01,2016-10-01,41,30,99,99,1,2016-11-01,0
17219285,1hapHdS7PNnKCRXnUp4X+5p//irxpsojYZB95dIndq0=,2017-01-01,2016-10-01,41,30,149,149,1,2016-10-31,0
17219291,1hapHdS7PNnKCRXnUp4X+5p//irxpsojYZB95dIndq0=,2017-01-01,2016-10-01,41,30,149,149,1,2016-11-30,0


In [68]:
count_transaction=grouped_train_transaction.agg({'payment_method_id':'count'}) 
# count_transaction.columns = count_transaction.columns.droplevel()
count_transaction.reset_index(inplace = True)
count_transaction.columns =['msno','count_transaction']

In [69]:
count_transaction

,msno,count_transaction
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,3
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,2
...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,2
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,3
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,3
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,4


In [51]:
agg_func = {'payment_method_id':[lambda x: stats.mode(x)[0][0]], # the mode of payment_method_id
            'payment_plan_days':['mean','sum'], # the mean of payment_plan_days
            'plan_list_price':['mean'], # the mean of payment_plan_days
            'actual_amount_paid':['mean','sum'], # the mean of actual_amount_paid
            'is_auto_renew':[lambda x: stats.mode(x)[0][0]], # the mode of is_auto_renew
            'is_cancel':[lambda x: stats.mode(x)[0][0],'sum']} # the mode of is_cancel or not
#             'amt_per_day':['mean'], # the mean of amount paid per day
#             'membership_duration':['mean'], # the mean of membership_duration
#             'registered_via':[lambda x: stats.mode(x)[0][0]], # the mode of the way of register
#             'autorenew_&_not_cancel':[lambda x: stats.mode(x)[0][0]], # the mode of autorenew_&_not_cancel
#             'notAutorenew_&_cancel':[lambda x: stats.mode(x)[0][0]]} # the mode of notAutorenew_&_cancel

In [ ]:
grouped_train_transaction = train_transaction.copy().groupby('msno')
train_t=grouped_train_transaction.agg(agg_func)

In [55]:
train_t

,msno,<lambda>,mean,sum,mean,mean,sum,<lambda>,<lambda_0>,sum
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395.0,395,1599.0,1599.0,1599,0,0,0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0
...,...,...,...,...,...,...,...,...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30.0,60,180.0,180.0,360,0,0,0
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30.0,90,149.0,149.0,447,1,0,1
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30.0,120,149.0,149.0,596,1,0,1


In [54]:
train_t.columns = train_t.columns.droplevel()
train_t.reset_index(inplace = True)

In [56]:
column_name=['msno','payment_method_id_mode','payment_plan_days_mean','payment_plan_days_sum','plan_list_price_mean','actual_amount_paid_mean',
             'actual_amount_paid_sum','is_auto_renew_mode','is_cancel_mode','is_cancel_sum']

In [57]:
train_t.columns=column_name

In [58]:
train_t

,msno,payment_method_id_mode,payment_plan_days_mean,payment_plan_days_sum,plan_list_price_mean,actual_amount_paid_mean,actual_amount_paid_sum,is_auto_renew_mode,is_cancel_mode,is_cancel_sum
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395.0,395,1599.0,1599.0,1599,0,0,0
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0
...,...,...,...,...,...,...,...,...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30.0,60,180.0,180.0,360,0,0,0
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30.0,90,149.0,149.0,447,1,0,1
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30.0,120,149.0,149.0,596,1,0,1


In [62]:
train_t['spend_per_day']=(train_t['actual_amount_paid_sum']/train_t['payment_plan_days_sum']).round(2)

In [63]:
train_t

,msno,payment_method_id_mode,payment_plan_days_mean,payment_plan_days_sum,plan_list_price_mean,actual_amount_paid_mean,actual_amount_paid_sum,is_auto_renew_mode,is_cancel_mode,is_cancel_sum,spend_per_day
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395.0,395,1599.0,1599.0,1599,0,0,0,4.05
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0,3.30
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0,4.97
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0,4.97
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0,4.97
...,...,...,...,...,...,...,...,...,...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30.0,60,180.0,180.0,360,0,0,0,6.00
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30.0,90,149.0,149.0,447,1,0,1,4.97
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0,4.97
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30.0,120,149.0,149.0,596,1,0,1,4.97


In [70]:
train_transaction = pd.merge(train_t, count_transaction, on = 'msno', how = 'inner')

In [72]:
train_transaction

,msno,payment_method_id_mode,payment_plan_days_mean,payment_plan_days_sum,plan_list_price_mean,actual_amount_paid_mean,actual_amount_paid_sum,is_auto_renew_mode,is_cancel_mode,is_cancel_sum,spend_per_day,count_transaction
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395.0,395,1599.0,1599.0,1599,0,0,0,4.05,1
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0,3.30,2
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2
...,...,...,...,...,...,...,...,...,...,...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30.0,60,180.0,180.0,360,0,0,0,6.00,2
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30.0,90,149.0,149.0,447,1,0,1,4.97,3
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30.0,120,149.0,149.0,596,1,0,1,4.97,4


In [73]:
members = pd.read_csv('members_v3.csv') 

In [75]:
train_member_transaction=pd.merge( user_test, members, on = 'msno', how = 'left')

In [80]:
train_member_transaction

,msno,membership_expire_date,date_prior_90,city,bd,gender,registered_via,registration_init_time,lifetume
0,PY9teeSI3d11//HOjAECAuA+zonRt/32J8YrEOQdk24=,2017-01-31,2016-10-31,1.0,0.0,NaN,7.0,2015-10-31,458 days
1,vSZ4aJziXuMmmm4wKMVTg3plvNspkVlJ1lZH1lsUd50=,2017-01-31,2016-10-31,13.0,37.0,female,9.0,2005-10-05,4136 days
2,/w9E+Tz8Ny3anC3hVFy8iLXGdWFoQ13fmwKjjd2daLg=,2017-01-31,2016-10-31,17.0,29.0,female,3.0,2014-02-25,1071 days
3,CTnwF1BSyfABgVDqKF526DsK+8jOSNdnCzsqLk+u9U4=,2017-01-31,2016-10-31,13.0,24.0,female,3.0,2013-03-30,1403 days
4,4ESDENnUYIB7kwWTS+sfNTEMq/LTfnDTkV6e9zA3MFg=,2017-01-31,2016-10-31,9.0,30.0,male,9.0,2008-10-28,3017 days
...,...,...,...,...,...,...,...,...,...
990740,rVsmR7Yq79ekFXc5urI2DHBjsPD5RsRinpPWkXvFkvM=,2017-01-01,2016-10-01,1.0,0.0,NaN,7.0,2015-09-02,487 days
990741,oy+BCqqCbA5Ea6SJLVDjzQt2Wz/zUY7NRmdGK3cvgE8=,2017-01-01,2016-10-01,5.0,30.0,male,7.0,2011-12-16,1843 days
990742,1Dpo2oTAeDBCWbCcLuj9uzMfJ8u0h2oaUcIOsV2zwNw=,2017-01-01,2016-10-01,5.0,43.0,male,9.0,2006-08-01,3806 days
990743,pJzUz1g5hC/tHN3/DZsjF0l8j5zb3Tykmc4HJMv98QY=,2017-01-01,2016-10-01,1.0,0.0,NaN,7.0,2015-08-02,518 days


In [77]:
train_member_transaction['registration_init_time']=pd.to_datetime(train_member_transaction['registration_init_time'], format='%Y%m%d')

In [97]:
train_member_transaction['lifetime']=train_member_transaction['membership_expire_date'].sub(train_member_transaction['registration_init_time'], axis=0)

In [98]:
train_data_transaction_member=pd.merge(train_transaction, train_member_transaction, on = 'msno', how = 'left')

In [99]:
train_data_transaction_member.drop(['date_prior_90','gender','registration_init_time'],1,inplace=True)

In [102]:
train_data_transaction_member

,msno,payment_method_id_mode,payment_plan_days_mean,payment_plan_days_sum,plan_list_price_mean,actual_amount_paid_mean,actual_amount_paid_sum,is_auto_renew_mode,is_cancel_mode,is_cancel_sum,spend_per_day,count_transaction,membership_expire_date,city,bd,registered_via,lifetime
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395.0,395,1599.0,1599.0,1599,0,0,0,4.05,1,2017-01-04,6.0,22.0,3.0,1472 days
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0,3.30,2,2017-01-15,1.0,0.0,7.0,952 days
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-19,15.0,26.0,9.0,2254 days
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2,2017-01-26,1.0,0.0,7.0,852 days
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2,2017-01-15,18.0,21.0,7.0,306 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30.0,60,180.0,180.0,360,0,0,0,6.00,2,2017-01-29,5.0,31.0,3.0,640 days
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30.0,90,149.0,149.0,447,1,0,1,4.97,3,2017-01-14,9.0,0.0,7.0,1340 days
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-24,4.0,48.0,9.0,958 days
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30.0,120,149.0,149.0,596,1,0,1,4.97,4,2017-01-23,1.0,0.0,7.0,1743 days


In [101]:
train_data_transaction_member

In [86]:
train_data_transaction_member.dropna(inplace=True)

In [107]:
train_data_transaction_member['lifetime'].dtype

dtype('<m8[ns]')

In [110]:
train_data_transaction_member['lifetime'].shape[0]

970086

In [1]:
# train_data_transaction_member['lifetime']=train_data_transaction_member['lifetime'].astype('timedelta64[D]')

In [119]:
train_data_transaction_member['lifetime']=train_data_transaction_member['lifetime'].fillna(0).astype(int)

In [123]:
train_data_transaction_member

,msno,payment_method_id_mode,payment_plan_days_mean,payment_plan_days_sum,plan_list_price_mean,actual_amount_paid_mean,actual_amount_paid_sum,is_auto_renew_mode,is_cancel_mode,is_cancel_sum,spend_per_day,count_transaction,membership_expire_date,city,bd,registered_via,lifetime
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395.0,395,1599.0,1599.0,1599,0,0,0,4.05,1,2017-01-04,6.0,22.0,3.0,1472
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0,3.30,2,2017-01-15,1.0,0.0,7.0,952
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-19,15.0,26.0,9.0,2254
3,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2,2017-01-26,1.0,0.0,7.0,852
4,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2,2017-01-15,18.0,21.0,7.0,306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970081,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,36,30.0,60,180.0,180.0,360,0,0,0,6.00,2,2017-01-29,5.0,31.0,3.0,640
970082,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30.0,90,149.0,149.0,447,1,0,1,4.97,3,2017-01-14,9.0,0.0,7.0,1340
970083,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-24,4.0,48.0,9.0,958
970084,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30.0,120,149.0,149.0,596,1,0,1,4.97,4,2017-01-23,1.0,0.0,7.0,1743


In [124]:
train_label=pd.read_csv('train.csv')

In [125]:
train=pd.merge(train_data_transaction_member,train_label,on='msno',how='left')

In [127]:
train.dropna(inplace=True)

In [135]:
train.reset_index(drop=True,inplace=True)

In [136]:
train

,msno,payment_method_id_mode,payment_plan_days_mean,payment_plan_days_sum,plan_list_price_mean,actual_amount_paid_mean,actual_amount_paid_sum,is_auto_renew_mode,is_cancel_mode,is_cancel_sum,spend_per_day,count_transaction,membership_expire_date,city,bd,registered_via,lifetime,is_churn
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30.0,60,99.0,99.0,198,1,0,0,3.30,2,2017-01-15,1.0,0.0,7.0,952,0.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-19,15.0,26.0,9.0,2254,0.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2,2017-01-26,1.0,0.0,7.0,852,0.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30.0,60,149.0,149.0,298,1,0,0,4.97,2,2017-01-15,18.0,21.0,7.0,306,0.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-23,22.0,32.0,9.0,1828,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787861,zzx7wcys2Nha6yP2Skjs+R1/JNAfEuAWUaAp0qsV2Cg=,41,30.0,90,99.0,99.0,297,1,0,1,3.30,3,2017-01-04,15.0,30.0,7.0,2297,1.0
787862,zzxPS8+wCuE4HE85EDp4WBHgS5l548vfwXJbQq1HZ9I=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-10,22.0,28.0,9.0,3441,0.0
787863,zzxZeMFx2fjfKZigMnJa2w0EmloDbm8+8nTf/o/00GY=,41,30.0,60,99.0,99.0,198,1,0,0,3.30,2,2017-01-26,4.0,24.0,7.0,1787,0.0
787864,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,30.0,90,149.0,149.0,447,1,0,0,4.97,3,2017-01-24,4.0,48.0,9.0,958,0.0


In [141]:
train.to_csv('train_transaction_member_lebel.csv',index=False)